<a href="https://colab.research.google.com/github/akan72/comp790/blob/master/notebooks/geometricTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pwd
#!git clone https://github.com/akan72/comp790.git

'C:\\Users\\lexokan\\Dropbox\\projects\\comp790\\notebooks'

In [12]:
cd comp790

[WinError 2] The system cannot find the file specified: 'comp790'
C:\Users\lexokan\Dropbox\projects\comp790\old\notebooks


In [5]:
import torch
import torch.nn
import torch.nn.functional as F

import torch_geometric.transforms as T

from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.data import DataLoader

from torch_geometric.data import InMemoryDataset
from torch_geometric.datasets import Planetoid, MNISTSuperpixels, TUDataset

In [14]:
dataset = Planetoid(root='../../data/geometric/CORA', name='Cora')

# dataset = ShapeNet(root='/tmp/ShapeNet', category='Airplane',
#                     pre_transform=T.KNNGraph(k=6))


In [15]:
print(len(dataset))

1


In [16]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(10):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [9]:
model.eval()
_, pred = model(data).max(dim=1)
correct = pred[data.test_mask].eq(data.y[data.test_mask]).sum().item()
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.7780


In [22]:
print(len(data.train_mask))

2708


In [21]:
print(len(data.test_mask))

2708
